In [17]:
#Import modules
import numpy as np

class Ring:
    def __init__(self, size):
        self.size = size
        self.ring = []
        self.sl = []

    def __str__(self):
        return str(self.ring)
        
    def addFilament(self, length):
        filPos = np.array(np.zeros(self.size), dtype = bool)
        start = np.random.randint(self.size)
        end = start + length
        if end > len(filPos):
            filPos[start:len(filPos)] = True
            filPos[0:end-len(filPos)] = True
        else:
            filPos[start:end] = True
        self.ring.append(filPos)
        return 1
       
    def sliceIt(self, pos, thickness):
        ring = np.array(self.ring)
        start = pos
        end = start + thickness
        if end > self.size:
            endPart = ring[:, start:self.size]
            startPart = ring[:, 0:end - self.size]
            sl = np.hstack((endPart,startPart))
        else:
            sl = ring[:, start:end]
        
        self.sl = sl
        
    def reportStats(self, verbose=True):
        sl = self.sl
        fils, thickness = sl.shape
        filament = np.array(np.ones(thickness), dtype=bool)
        nofilament = np.array(np.zeros(thickness), dtype=bool)
        completeFils = 0
        ends = 0
        for i in range(fils):
            if np.array_equal(sl[i], filament):
                completeFils += 1
            elif not np.array_equal(sl[i], nofilament):
                ends += 1
        if verbose:
            print('Numer of true filaments in the ring: {}'.format(len(self.ring)))
            print('Numer of filaments throught slice: {}'.format(completeFils))
            print('Numer of filaments ending in the slice: {}'.format(ends))
        return completeFils, ends

def simulate(ringSize, numFils, filLength):
    R = Ring(ringSize)
    for i in range(numFils):
        R.addFilament(filLength)
    return R

def makeStats(R, thickness, trials, verbose = False):
    endStat = []
    fils = []
    starts = []
    for i in range(trials):
        start = np.random.randint(R.size)
        R.sliceIt(start,thickness)
        fil, ends = R.reportStats(False)
        endStat.append(ends)
        fils.append(fil)
        starts.append(start)
        if verbose:
            print('Sliced at: {} - {}'.format(start, R.sl))

    endStat = np.array(endStat)
    fils = np.array(fils)

    print('Results:')
    print('Number of ends (simul): {}'.format(endStat.mean()))
    print('Number of observed filaments: {}'.format(fils.mean()))
    
    return fils.mean(), endStat.mean(), 


In [14]:
D = 1000
Nf = 500
L = 50
S = 30
Ne = 2 * Nf * (S - 1) / D   #4 * Nf * D / ( D - S )
No = Nf * (L - ( S - 1))/ D
print('Number of ends (theory) : {}'.format(Ne))
print('Number of observed filaments (theory) : {}'.format(No))
R = simulate(D, Nf, L)

Number of ends (theory) : 29.0
Number of observed filaments (theory) : 10.5


In [21]:
No_stat, Ne_stat = makeStats(R, S, 10)

Results:
Number of ends (simul): 29.2
Number of observed filaments: 10.5


In [ ]:
print(R)

## How does it work?

Let's say there is a container of *D*=10 positions and a filament with length *L*=5 randomly placed in the container. In this example, the filament starts at the position 3 of the container:

```
Container                 = |---|---|---|---|---|---|---|---|---|---|
Filament                  =         |---|---|---|---|---|
```

Let's now place the filament in the container and for that, we will mark the positions in the container with the filament with T and without the filament with F.

```
Container with filament:  = |-F-|-F-|-T-|-T-|-T-|-T-|-T-|-F-|-F-|-F-|
```

Now, let's suppose that we can only look at *S*=3 positions at once in this container, which we will call *slice*. The question is:

**What is the probability of finding one of the ends of the filament?**

>Before we continue, we will define the regime where $ S << L << D $. In other words, the size of the slice is much smaller than the size of the filament which is much smaller than the size of the container

Probabilities is, in general, the ration between the number of possibilities of what you want and the number of *all* possibilities.

Because the container is "connected" from the end to the begining, the number of *all* possible slices, independently of the number of slice and it is the length of the the container *D*.

Now, let's consider that we see an end when there is a switch from **T** to **F** or **F** to **T** within the 2 positions we are allowed to look at it. In this example the two ends of the filament are in the position 3 and position 7. Let's count how many possible slices os *S*=3 can capture at least one of them:

|slice position|pos 1 | pos 2 | pos 3 | end |
|:-:|:-: |:-:|:-:|
| 1 | F |F| T| Yes|
| 2 | F |T| T| Yes|
| 3 | T |T| T| No |
| 4 | T |T| T| No |
| 5 | T |T| T| No |
| 6 | T |T| F| Yes|
| 7 | T |F| F| Yes|
| 8 | F |F| F| No |
| 9 | F |F| F| No |
| 10| F |F| F| No |

There are two possible slices, the slice at 1 and 2, related to the left terminus of the filament and the two others, slices 6 and 7, related to the right terminus. 
Thus, the probability of finding one end in a slice of 3 positions is: $4/10$ = 0.25. Notice that this is the contribution of 2 positions for each end.

For the sake of clarity, let's increase the size of the slice to 4. *S*=4:

|slice position|pos 1 | pos 2 | pos 3 | pos 4 | end |
|:-:|:-: |:-:|:-:|:-:|
| 1 | F |F| T| T| Yes|
| 2 | F |T| T| T| Yes|
| 3 | T |T| T| T| No |
| 4 | T |T| T| T| No |
| 5 | T |T| T| F| Yes |
| 6 | T |T| F| F| Yes|
| 7 | T |F| F| F| Yes|
| 8 | F |F| F| F| No |
| 9 | F |F| F| F| No |
| 10| F |F| F| T| Yes |

Now we have three possible slices for the left terminus:10, 1 and 2. Also we have three spossible slices for the right terminus: and 5,6 and 7. Which gives us 6 possibilities and thus, the probability of finding one end in a slice of 4 positions is: $6/10$ = 0.25. Notice that from the tota of 6 possibilities, 3 comes from each end.

Based on these results in this discrete example, it is easily to see that the number of possible slices that contain one of the size of the slice minus 1 for each end. Just to make it even more clear... let's look at one of the ends:

```
Position:                     1   2   3   4   5   6   7   8   9   10   1
Container with filament:  = |-F-|-F-|-T-|-T-|-T-|-T-|-T-|-F-|-F-|-F-||-F-|
slice 7 - no end          =             |---|---|---|---|
slice 5                   =                 |---|---|---|---|
slice 6                   =                     |---|---|---|---|
slice 7                   =                         |---|---|---|---|
slice 8 - no end          =                             |---|---|---||---|
```

Finally, we can confidently say that the probability of finding one of the ends of the filament of length *L* randomnly placed in a container of size *D* with a slice of size *S* is:

$$ P_{e} = \frac{2 (S - 1)}{D}$$

where the **2** is because there are two ends in a filament.

To generalize this to more than one filament, and assuming that their placement do not affect the placement of other filaments, one just need to multiply it by the number of filaments. So, given the number of filaments, the size of the container and the size of the slice, one can predict the expected value of ends in a single slice with:

$$ \langle N_{e} \rangle = \frac{2N_{f}(S - 1)}{D}$$

For example, in a compartiment of the size 1000 and 500 filaments with a slice of 3 you can expect to see on average 2 ends of filements. However, if you increase the slice to 30 under the same conditions, you are suppose to expect to see 30 ends.

***

The other important question is:

**What is the probability of finding a filament running continously through the slice?**

Using the same example above let's examine the number of possible slices, first with *S*=3, that we find a continuous filament:

|slice position|pos 1 | pos 2 | pos 3 | filament |
|:-:|:-: |:-:|:-:|
| 1 | F |F| T| No|
| 2 | F |T| T| No|
| 3 | T |T| T| Yes |
| 4 | T |T| T| Yes |
| 5 | T |T| T| Yes |
| 6 | T |T| F| No|
| 7 | T |F| F| No|
| 8 | F |F| F| No |
| 9 | F |F| F| No |
| 10| F |F| F| No |

In this case, we had three possibilities in 10.

increasing the slice to *S*=4:

|slice position|pos 1 | pos 2 | pos 3 | pos 4 | end |
|:-:|:-: |:-:|:-:|:-:|
| 1 | F |F| T| T| No|
| 2 | F |T| T| T| No|
| 3 | T |T| T| T| Yes |
| 4 | T |T| T| T| Yes |
| 5 | T |T| T| F| No |
| 6 | T |T| F| F| No|
| 7 | T |F| F| F| No|
| 8 | F |F| F| F| No |
| 9 | F |F| F| F| No |
| 10| F |F| F| T| No |

Now we have two possibilities in 10. 

In other words, the larger the slice, less chances to find a filament that runs through the entire slice. Let's try to visualize why. First with the *S*=3:


```
Position:                     1   2   3   4   5   6   7   8   9   10   1
Container with filament:  = |-F-|-F-|-T-|-T-|-T-|-T-|-T-|-F-|-F-|-F-||-F-|
slice 2 - no thourgh fil  =     |-X-|---|---|
slice 3                   =         |---|---|---|
slice 4                   =             |---|---|---|
slice 5                   =                 |---|---|---|
slice 6 - no through fil  =                     |---|---|-X-|
```

and with *S*=4

```
Position:                     1   2   3   4   5   6   7   8   9   10   1
Container with filament:  = |-F-|-F-|-T-|-T-|-T-|-T-|-T-|-F-|-F-|-F-||-F-|
slice 2 - no through fil  =     |-X-|---|---|---|
slice 3                   =         |---|---|---|---|
slice 4                   =             |---|---|---|---|
slice 5   no through fil  =                 |---|---|---|-X-|
slice 6 - no through fil  =                     |---|---|-X-|-X-|
```

From that, we infer that the number of possible slices with a filament completely running through depends on the length of the filament minus the size of the slice minus 1. Thus the probability of spoting a filament running through a slice can be writen as:

$$ P_{fil} = \frac{L - (S - 1)}{D}$$

Similarly with the $N_{e}$, the expected number of **o**bserved filaments $\langle N_{o} \rangle $ can be calculated as: 

$$\langle N_{o} \rangle = N_{fil}\frac{L - (S - 1)}{D}$$

In other words, in the same scenario with a container of 1000, 500 filaments and a slice of 3, we should expect to find **24** filaments running continously through the slice. In contrast, increasing the slice size to 30, should decrease the number of filaments running through to **10.5**.

## Let's find out how to use these to make predictions.

The major important question is: 

#### Can we estimatethe the length L of the filaments and the total number of filaments by observing the number of ends and the numbers of filaments running through a slice of a known size in a container of a known size?

In other words:

| knowns | unknowns |
|:-:|:-:|
| D, S, $ \langle N_{o} \rangle $, $ \langle N_{e} \rangle $ | $N_{f}$, L |

Using the two equations derived earlier:

$$ \langle N_{e} \rangle = 2 N_{f} (S - 1) / D $$


$$\langle N_{o} \rangle = N_{f}\frac{L - (S - 1)}{D}$$

manipulating it we can recover:

$$ L_{est} = (S-1)[1 + \frac{\langle N_{o} \rangle}{\langle N_{e} \rangle}] $$

$$ {N_{f}}_{est} = \frac{D \langle N_{e} \rangle}{ 2 (S - 1)} $$

In [34]:
L_est = (S - 1) * (1 + 2 * No_stat / Ne_stat)
Nf_est = Ne_stat * D / (2 * (S - 1) )
print('Estimated length (L = {}): {:.2f}'.format(L, L_est) )
print('Estimated number of filaments (Nf = {}): {:0.2f}'.format(Nf, Nf_est))

Estimated length (L = 50): 49.86
Estimated number of filaments (Nf = 500): 503.45


In [103]:
np.array_equal(R.sl[1], nofilament)

False